### Comparison and Prediction Results

In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16_input

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 #### Load both the models

In [4]:
resnet50 = keras.models.load_model('classifier_resnet_model.h5')
vgg16 = keras.models.load_model('classifier_vgg16_model.h5')
print(resnet50.summary())
print(vgg16.summary())



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
None
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                 

#### Data Generation for testing

In [6]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [31]:
data_generator_resnet = ImageDataGenerator(
    preprocessing_function=preprocess_resnet50_input,
)

test_resnet_generator = data_generator_resnet.flow_from_directory(
    'concrete_data/validation',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

data_generator_vgg = ImageDataGenerator(
    preprocessing_function=preprocess_vgg16_input,
)

test_vgg_generator = data_generator_vgg.flow_from_directory(
    'concrete_data/validation',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 7243 images belonging to 2 classes.
Found 7243 images belonging to 2 classes.


#### Evaluating the models

In [32]:
result_resnet = resnet50.evaluate_generator(
    test_resnet_generator,
    verbose=1)

C:\Users\user\AppData\Local\Temp/ipykernel_22672/1033990696.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  result_resnet = resnet50.evaluate_generator(


73/73 [==============================] - 322s 4s/step - loss: 0.0030 - accuracy: 0.9994


In [33]:
result_vgg16 = vgg16.evaluate_generator(
    test_vgg_generator,
    verbose=1)

C:\Users\user\AppData\Local\Temp/ipykernel_22672/327617202.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  result_vgg16 = vgg16.evaluate_generator(


73/73 [==============================] - 452s 6s/step - loss: 0.0170 - accuracy: 0.9957


VGG16 Performance

In [34]:
print(f"Accuracy: {result_vgg16[1]}")
print(f"Loss: {result_vgg16[0]}")

Accuracy: 0.9957200288772583
Loss: 0.016984829679131508


ResNet50 Performance

In [35]:
print(f"Accuracy: {result_resnet[1]}")
print(f"Loss: {result_resnet[0]}")

Accuracy: 0.999447762966156
Loss: 0.003016134724020958


Make predictions

In [36]:
import numpy as np

def get_predictions(predictions):
    for pred in predictions:
        position = np.argmax(pred)
        if position == 1:
            print("Positive")
        if position == 0:
            print("Negative")

In [37]:
resnet_prediction = resnet50.predict_generator(
    test_resnet_generator,
    verbose=1)

C:\Users\user\AppData\Local\Temp/ipykernel_22672/2491213823.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  resnet_prediction = resnet50.predict_generator(


73/73 [==============================] - 288s 4s/step


In [ ]:
vgg16_prediction = vgg16.predict_generator(
    test_vgg_generator,
    verbose=1)

C:\Users\user\AppData\Local\Temp/ipykernel_22672/1315888069.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg16_prediction = vgg16.predict_generator(


42/73 [================>.............] - ETA: 2:59

Class predictions report

In [41]:
get_predictions(resnet_prediction[:5])

Positive
Positive
Negative
Positive
Positive


In [42]:
get_predictions(vgg16_prediction[:5])

Negative
Negative
Positive
Positive
Negative
